In [20]:
import pandas as pd

In [21]:
candy_data = pd.read_csv('candy-data.csv')
print(candy_data.describe())
print(candy_data.head())

       chocolate     fruity    caramel  peanutyalmondy     nougat  \
count  85.000000  85.000000  85.000000       85.000000  85.000000   
mean    0.435294   0.447059   0.164706        0.164706   0.082353   
std     0.498738   0.500140   0.373116        0.373116   0.276533   
min     0.000000   0.000000   0.000000        0.000000   0.000000   
25%     0.000000   0.000000   0.000000        0.000000   0.000000   
50%     0.000000   0.000000   0.000000        0.000000   0.000000   
75%     1.000000   1.000000   0.000000        0.000000   0.000000   
max     1.000000   1.000000   1.000000        1.000000   1.000000   

       crispedricewafer       hard        bar   pluribus  sugarpercent  \
count         85.000000  85.000000  85.000000  85.000000     85.000000   
mean           0.082353   0.176471   0.247059   0.517647      0.478647   
std            0.276533   0.383482   0.433861   0.502654      0.282778   
min            0.000000   0.000000   0.000000   0.000000      0.011000   
25%     

In [24]:
#como tenemos gran parte de variables categorícas realizaremos una regressión logística que nos ayude a diferenciar
#entre caramelos ganadores y caramelos perdedores.
def pricequantiles(x):
    if x>=0 and x<=0.222:
        return 0
    elif x>0.222 and x<=0.465:
        return 1
    elif x>0.465 and x<=0.651:
        return 2
    elif x>0.651:
        return 3
    
def sugarquantiles(x):
    if x>=0 and x<=0.255:
        return 0
    elif x>0.255 and x<=0.465:
        return 1
    elif x>0.465 and x<=0.732000:
        return 2
    elif x>0.732000:
        return 3
    
def pricemedium(x):
    if x<=0.468882:
        return 0
    else:
        return 1

def sugarmedium(x):
    if x<=0.478647:
        return 0
    else:
        return 1 
def winpercent(x):
    if x<50:
        return 0
    elif x>=50:
        return 1

candy_data['pricequantiles'] = candy_data['pricepercent'].apply(lambda x:pricequantiles(x))
candy_data['pricemedium'] = candy_data['pricepercent'].apply(lambda x: pricemedium(x))
candy_data['sugarquantiles'] = candy_data['sugarpercent'].apply(lambda x:sugarquantiles(x))
candy_data['sugarmedium'] = candy_data['sugarpercent'].apply(lambda x:sugarmedium(x))
candy_data['looseorwin'] = candy_data['winpercent'].apply(lambda x:winpercent(x))

In [26]:
#dividimos las variables x e y.
X_values = candy_data.filter(['chocolate','fruity','caramel','peanutyalmondy'
                           'nougat','crispedricewafer','hard','bar'
                           'pluribus','pricemedium','sugarmedium'], axis=1).values.tolist()
print(X_values[:10])
Y_values = candy_data.filter(['looseorwin'], axis=1).values.tolist()
print(Y_values[:10])


[[1, 0, 1, 1, 0, 1, 1], [1, 0, 0, 0, 0, 1, 1], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0], [1, 0, 1, 0, 0, 1, 1], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 1]]
[[1], [1], [0], [0], [1], [1], [1], [0], [0], [0]]


In [31]:
#dividimos el dataset en train y en test, usamos el argumento stratify para tener la misma proporcion de ganador y perdedor
#entre nuestra conjunto de test y train. Ademas el test representara el 25% del total.
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_values, Y_values, stratify=Y_values, test_size=0.25)
print(X_train[:10])
print(X_test[:10])
print(Y_train[:10])
print(Y_test[:10])

[[1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 1, 0, 1], [0, 1, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0], [1, 0, 1, 0, 0, 1, 1], [1, 0, 1, 1, 0, 1, 1], [0, 1, 0, 0, 1, 1, 1], [1, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 1, 0]]
[[0, 1, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 1, 0, 1, 1], [0, 0, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 0, 0]]
[[0], [0], [0], [1], [1], [1], [0], [0], [0], [1]]
[[0], [1], [1], [1], [0], [0], [0], [0], [1], [0]]


In [51]:
#ajusta,os el modelo de regresion logistica.
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
res = model.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[[ 1.20460857  0.00491553 -0.00991309  0.61911835 -1.40869348  0.39303074
   0.82274058]]


//anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [61]:
#realizamos el contraste para el test. y comprobamos el ajuste con el me
from sklearn.metrics import  accuracy_score
model_prediction = res.predict(X_test)
score = accuracy_score(Y_test,model_prediction)
print("Calidad del ajuste de la regresión logística {}".format(score))

Calidad del ajuste de la regresión logística 0.7272727272727273
